In [1]:
import os
import numpy as np
import scipy.constants as sc


In [3]:
def com_correction(force_k_data,volume,T,fu):
    kb = sc.value('Boltzmann constant') # kb in SI base units
    kb_kcal = sc.value('Boltzmann constant in eV/K') * 23.06054 # kb in kcal / mol K
    mu = sc.value('atomic mass constant')
    hbar = sc.value('Planck constant over 2 pi') # hbar in SI base units
    Na = sc.value('Avogadro constant') # Avagadro's number
    
    F_harm = 0.0
    natoms = 0
    total_mass = 0.0
    com_correction_data = []
    
    for ty, num_type, mass, force_k, msd in force_k_data:
            
        natoms += num_type

        total_mass += mass * num_type
        com_correction_data.append((num_type, mass, force_k)) # this is list data used to compute COM correction below
                
    mass_term = 0.0
    total_mass_squared = total_mass**2

    for num_type, mass, force_k in com_correction_data:

        mass_term += (num_type * (mass**2))/(total_mass_squared * force_k)

    F_cmco = (1.5 * kb_kcal * T) * np.log((2 * np.pi * kb_kcal * T) * mass_term)/fu + (kb_kcal * T * np.log(fu/volume))

    return F_cmco



In [8]:
root_directory = '/project2/andrewferguson/mjianming/project_cation_control-midway3/project_cation_control/cation_screening/TMA/2D/'

directories = [d for d in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, d)) and not d.startswith('.')]

# clear the file first
with open(root_directory+'/corrections.dat', 'w') as f1:
    f1.truncate(0)  # The file is cleared by truncating it to 0 bytes

for dirname in directories:
    if not dirname.startswith('.') and (dirname!='integrate_tools') and os.path.isdir(os.path.join(root_directory, dirname)):
        directory_path = os.path.join(root_directory, dirname)
        with open(directory_path+"/volume.dat", 'r') as file:
            volume = float(file.read())
        with open(directory_path+"/fu.dat", 'r') as file:
            fu = float(file.read())
        T = 300 # K       
        force_k_data = np.genfromtxt(directory_path+"/force_k.dat", delimiter = '')
        A_com = com_correction(force_k_data,volume,T,fu)
        A_com = round(A_com, 3)
        with open(root_directory+'/corrections.dat','a') as f:
            f.write(f'Topology A_com formula_unit \n')
            f.write(f'{dirname} {str(A_com)} {int(fu)} \n')
            
